# P&L Attribution Analysis

This notebook performs P&L decomposition for a Bitcoin call option, breaking down price changes into their theoretical Greek drivers (Delta, Gamma, Theta, Vega).

In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from scipy.stats import norm

In [ ]:
# Configuration
BASE_URL = "https://thalex.com/api/v2/public"

timestamps = (
    int(dt.datetime(2025, 11, 24, 8, 0).timestamp()),
    int(dt.datetime(2025, 12, 3, 8, 0).timestamp()),
)

underlying = 'BTC'
strike = 100_000
expiry = '26DEC25'
instrument_name = f"{underlying}-{expiry}-{strike}-C"
resolution = "1m"

dfs = {}

In [ ]:
# Fetch option mark price data
endpoint = "mark_price_historical_data"
url = f"{BASE_URL}/{endpoint}"

COLUMNS = [
    "ts",
    "mark_price_open",
    "mark_price_high",
    "mark_price_low",
    "mark_price_close",
    "iv_open",
    "iv_high",
    "iv_low",
    "iv_close",
    "tob",
]

dfs["mark"] = pd.DataFrame(
    requests.get(
        url,
        params={
            "from": timestamps[0],
            "to": timestamps[1],
            "resolution": resolution,
            "instrument_name": instrument_name,
        },
    )
    .json()
    .get("result")
    .get("mark"),
    columns=COLUMNS,
).assign(
    date_time=lambda df: pd.to_datetime(df["ts"], unit="s"),
)

dfs["mark"]

In [ ]:
# Fetch instrument details
endpoint = "instrument"
url = f"{BASE_URL}/{endpoint}"

instrument = (
    requests.get(
        url,
        params={
            "instrument_name": instrument_name,
        },
    )
    .json()
    .get("result")
)

instrument

In [ ]:
# Fetch underlying future data
endpoint = "mark_price_historical_data"
url = f"{BASE_URL}/{endpoint}"

dfs["future"] = pd.DataFrame(
    requests.get(
        url,
        params={
            "instrument_name": f"{underlying}-{expiry}",
            "resolution": resolution,
            "from": timestamps[0],
            "to": timestamps[1],
        },
    )
    .json()
    .get("result")
    .get("mark"),
    columns=[
        "ts",
        "mark_price_open",
        "mark_price_high",
        "mark_price_low",
        "mark_price_close",
        'tob'
    ],
)

dfs["future"]

In [ ]:
# Greek calculation functions
def calc_delta(fw, strike, tte_seconds, iv, option_type="call"):
    tau = tte_seconds / (365.25 * 24 * 60 * 60)
    d1 = (np.log(fw / strike) + 0.5 * iv**2 * tau) / (iv * np.sqrt(tau))
    if option_type == "call":
        return norm.cdf(d1)
    return norm.cdf(d1) - 1


def calc_vega(fw, strike, tte_seconds, iv):
    tau = tte_seconds / (365.25 * 24 * 60 * 60)
    d1 = (np.log(fw / strike) + 0.5 * iv**2 * tau) / (iv * np.sqrt(tau))
    return fw * norm.pdf(d1) * np.sqrt(tau)


def calc_gamma(fw, strike, tte_seconds, iv) -> float:
    tau = tte_seconds / (365.25 * 24 * 60 * 60)
    d1 = (np.log(fw / strike) + 0.5 * iv**2 * tau) / (iv * np.sqrt(tau))
    return norm.pdf(d1) / (fw * iv * np.sqrt(tau))


def calc_theta(fw, strike, tte_seconds, iv):
    tau = tte_seconds / (365.25 * 24 * 60 * 60)
    d1 = (np.log(fw / strike) + 0.5 * iv**2 * tau) / (iv * np.sqrt(tau))
    theta_annual = -(fw * norm.pdf(d1) * iv) / (2 * np.sqrt(tau))
    return theta_annual / 365.25

In [ ]:
# P&L Attribution calculation
strike_price = instrument.get("strike_price")
option_type = instrument.get("option_type", "call")
expiration_ts = instrument.get("expiration_timestamp")


def calc_greeks_and_pnl(df, pos=1):
    df["tte_seconds"] = expiration_ts - df["ts"]

    df["delta"] = df.apply(
        lambda row: calc_delta(
            row["mark_future_price_close"],
            strike_price,
            row["tte_seconds"],
            row["iv_close"],
            option_type,
        ),
        axis=1,
    )
    df["gamma"] = df.apply(
        lambda row: calc_gamma(
            row["mark_future_price_close"],
            strike_price,
            row["tte_seconds"],
            row["iv_close"],
        ),
        axis=1,
    )
    df["theta"] = df.apply(
        lambda row: calc_theta(
            row["mark_future_price_close"],
            strike_price,
            row["tte_seconds"],
            row["iv_close"],
        ),
        axis=1,
    )
    df["vega"] = df.apply(
        lambda row: calc_vega(
            row["mark_future_price_close"],
            strike_price,
            row["tte_seconds"],
            row["iv_close"],
        ),
        axis=1,
    )

    df["dS"] = df["mark_future_price_close"] - df["mark_future_price_open"]
    df["dT"] = df["ts"].diff() / (24 * 60 * 60)
    df["dIV"] = df["iv_close"] - df["iv_open"]

    df["PL"] = df["mark_price_close"] - df["mark_price_open"]
    df["delta_PL"] = df["delta"].shift(1) * df["dS"]
    df["gamma_PL"] = df["gamma"].shift(1) * df["dS"] ** 2 * 0.5
    df["theta_PL"] = df["theta"].shift(1) * df["dT"]
    df["vega_PL"] = df["vega"].shift(1) * df["dIV"]

    df["attributed_PL"] = (
        df["delta_PL"] + df["gamma_PL"] + df["theta_PL"] + df["vega_PL"]
    )
    df["residual_PL"] = df["PL"] - df["attributed_PL"]
    df["gamma_theta_PL"] = df["gamma_PL"] + df["theta_PL"]

    return df


dfs["pnl"] = pd.merge(
    dfs["mark"], 
    dfs["future"].rename(columns={
        'mark_price_close': 'mark_future_price_close', 
        'mark_price_open': 'mark_future_price_open'
    }), 
    on="ts", 
    how="left"
).pipe(calc_greeks_and_pnl)

dfs["pnl"]

In [ ]:
# Visualization
df = dfs["pnl"].copy()

cols = [
    "delta_PL",
    "gamma_PL",
    "theta_PL",
    "vega_PL",
    "residual_PL",
    'PL'
]

for col in cols:
    df[f'{col}_cum'] = df[col].cumsum()

plt.figure(figsize=(16, 9))
plt.fill_between(df["date_time"], df["PL_cum"], color="#94b3fd", alpha=0.35, label="Total P&L")

for col in cols:
    plt.plot(df["date_time"], df[f'{col}_cum'], label=col.replace("_PL", ""))

plt.title("Total P&L vs Component Contributions (Cumulative)")
plt.xlabel("Time")
plt.ylabel("P&L")
plt.legend()
plt.tight_layout()
plt.show()